<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/PVT/oilassay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oil Assay Characterisation in NeqSim

This notebook demonstrates the use of the `OilAssayCharacterisation` class in NeqSim for petroleum fluid characterization from assay data.

## Introduction to Oil Assays

Oil assays are comprehensive analytical reports that characterize crude oil and petroleum products by their physical and chemical properties. They provide essential data for:

- **Refinery Planning**: Determining optimal processing strategies
- **Process Simulation**: Modeling distillation columns, separators, and other equipment
- **Product Quality Control**: Ensuring specifications are met
- **Economic Evaluation**: Assessing crude oil value and processing costs

### Key Assay Properties

1. **Distillation Curves**: Boiling point distributions (TBP - True Boiling Point)
2. **Density/API Gravity**: Fluid density characteristics
3. **Composition**: Hydrocarbon fractions and heteroatom content
4. **Physical Properties**: Viscosity, pour point, flash point

### Real-World Application

Equinor regularly publishes crude oil assay data for their production streams. For example, see the recent [Crude Oil Assay Update (October 2025)](https://www.equinor.com/news/crude-oil-assay-update-10-10-2025) which provides detailed characterization data for various North Sea crude oils.

## NeqSim OilAssayCharacterisation Class

The `OilAssayCharacterisation` class in NeqSim provides a systematic approach to:
- Convert assay data into thermodynamic pseudo-components
- Handle different data formats (mass %, volume %, API gravity)
- Apply proper characterization correlations
- Integrate with NeqSim's thermodynamic systems

In [ ]:
// Import Required Libraries and Classes
%jars ../../../target/neqsim-*.jar

import neqsim.thermo.characterization.OilAssayCharacterisation;
import neqsim.thermo.characterization.OilAssayCharacterisation.AssayCut;
import neqsim.thermo.system.SystemInterface;
import neqsim.thermo.system.SystemSrkEos;

System.out.println("NeqSim libraries imported successfully!");

## Create a Thermodynamic System

First, we need to create a thermodynamic system that will serve as the foundation for our oil characterization. We'll use the SRK equation of state which is commonly used for petroleum applications.

In [ ]:
// Create a thermodynamic system using SRK equation of state
// Standard conditions: 15°C (288.15 K) and 1 atm (1.01325 bar)
SystemInterface thermoSystem = new SystemSrkEos(288.15, 1.01325);

// Set mixing rule for hydrocarbon systems
thermoSystem.setMixingRule("classic");

// Create database for component properties
thermoSystem.createDatabase(true);

System.out.println("Thermodynamic system created:");
System.out.println("Temperature: " + thermoSystem.getTemperature() + " K");
System.out.println("Pressure: " + thermoSystem.getPressure() + " bar");
System.out.println("EOS: " + thermoSystem.getClass().getSimpleName());

## Understanding Oil Assay Data Structure

The `OilAssayCharacterisation` class uses the nested `AssayCut` class to represent individual petroleum fractions. Each cut can be defined with various properties:

- **Name**: Identifier for the fraction (e.g., "Light Naphtha", "Kerosene")
- **Mass/Volume Fractions**: Proportion of the cut in the total oil
- **Density or API Gravity**: Physical density characteristics  
- **Boiling Point**: Average boiling temperature of the fraction
- **Molar Mass**: Molecular weight (optional - can be calculated)

Let's explore the `AssayCut` class structure:

In [ ]:
// Create an AssayCut to demonstrate the structure
AssayCut exampleCut = new AssayCut("Example Cut");

System.out.println("AssayCut name: " + exampleCut.getName());
System.out.println("Has mass fraction: " + exampleCut.hasMassFraction());
System.out.println("Has volume fraction: " + exampleCut.hasVolumeFraction());
System.out.println("Has molar mass: " + exampleCut.hasMolarMass());

// The AssayCut uses a fluent API for easy configuration
System.out.println("\nThe AssayCut class provides fluent methods for configuration:");
System.out.println("- withMassFraction(double) / withWeightPercent(double)");
System.out.println("- withVolumeFraction(double) / withVolumePercent(double)");
System.out.println("- withDensity(double) / withApiGravity(double)");
System.out.println("- withAverageBoilingPointKelvin/Celsius/Fahrenheit(double)");
System.out.println("- withMolarMass(double)");

## Create Basic Assay Cuts with Mass Fractions

Let's create some basic petroleum fractions using mass fractions. This is the most straightforward way to define assay cuts when you have weight percentage data from laboratory analysis.

In [ ]:
// Create assay cuts using mass fractions
AssayCut lightFraction = new AssayCut("Light Naphtha")
    .withMassFraction(0.15)  // 15% by mass
    .withApiGravity(65.0)    // Light crude characteristics
    .withAverageBoilingPointCelsius(85.0);

AssayCut mediumFraction = new AssayCut("Kerosene")
    .withWeightPercent(25.0)  // Alternative way: 25% by weight
    .withApiGravity(45.0)
    .withAverageBoilingPointCelsius(200.0);

AssayCut heavyFraction = new AssayCut("Gas Oil")
    .withMassFraction(0.35)   // 35% by mass
    .withApiGravity(30.0)     // Heavier characteristics
    .withAverageBoilingPointCelsius(350.0);

AssayCut residueFraction = new AssayCut("Residue")
    .withMassFraction(0.25)   // 25% by mass (total = 100%)
    .withApiGravity(15.0)     // Heavy residue
    .withAverageBoilingPointCelsius(520.0);

System.out.println("Created assay cuts:");
System.out.println("Light Naphtha: " + (lightFraction.getMassFraction() * 100) + "% mass");
System.out.println("Kerosene: " + (mediumFraction.getMassFraction() * 100) + "% mass");
System.out.println("Gas Oil: " + (heavyFraction.getMassFraction() * 100) + "% mass");
System.out.println("Residue: " + (residueFraction.getMassFraction() * 100) + "% mass");

// Verify total mass fractions
double totalMass = lightFraction.getMassFraction() + mediumFraction.getMassFraction() +
                   heavyFraction.getMassFraction() + residueFraction.getMassFraction();
System.out.println("Total mass fraction: " + totalMass);

## Create Assay Cuts with Volume Fractions

Sometimes assay data is provided in volume percentages rather than mass percentages. The `OilAssayCharacterisation` class can handle this by converting volume fractions to mass fractions using density data.

In [ ]:
// Create assay cuts using volume fractions
AssayCut lightVolumeCut = new AssayCut("Light Distillate")
    .withVolumePercent(18.0)   // 18% by volume
    .withDensity(0.72)         // g/cm³ - light hydrocarbon density
    .withAverageBoilingPointCelsius(120.0);

AssayCut mediumVolumeCut = new AssayCut("Middle Distillate")
    .withVolumeFraction(0.30)  // 30% by volume (fraction form)
    .withDensity(0.82)         // g/cm³
    .withAverageBoilingPointCelsius(280.0);

AssayCut heavyVolumeCut = new AssayCut("Heavy Distillate")
    .withVolumePercent(32.0)   // 32% by volume
    .withDensity(0.92)         // g/cm³ - heavier hydrocarbons
    .withAverageBoilingPointCelsius(420.0);

AssayCut bottomsVolumeCut = new AssayCut("Bottoms")
    .withVolumePercent(20.0)   // 20% by volume
    .withDensity(1.05)         // g/cm³ - very heavy
    .withAverageBoilingPointCelsius(580.0);

System.out.println("Volume-based assay cuts:");
System.out.println("Light Distillate: " + (lightVolumeCut.getVolumeFraction() * 100) + "% volume, density = " + lightVolumeCut.resolveDensity() + " g/cm³");
System.out.println("Middle Distillate: " + (mediumVolumeCut.getVolumeFraction() * 100) + "% volume, density = " + mediumVolumeCut.resolveDensity() + " g/cm³");
System.out.println("Heavy Distillate: " + (heavyVolumeCut.getVolumeFraction() * 100) + "% volume, density = " + heavyVolumeCut.resolveDensity() + " g/cm³");
System.out.println("Bottoms: " + (bottomsVolumeCut.getVolumeFraction() * 100) + "% volume, density = " + bottomsVolumeCut.resolveDensity() + " g/cm³");

// Note: Mass fractions will be calculated automatically when apply() is called

## Working with API Gravity and Density Conversions

API gravity is a common measure in the petroleum industry. The class automatically converts between API gravity and density using the standard formula. Let's demonstrate this conversion capability.

In [ ]:
// Demonstrate API gravity to density conversion
AssayCut apiGravityCut = new AssayCut("API Gravity Example")
    .withMassFraction(0.20)
    .withApiGravity(35.0)  // Typical medium crude oil
    .withAverageBoilingPointCelsius(250.0);

AssayCut densityCut = new AssayCut("Direct Density Example")
    .withMassFraction(0.20)
    .withDensity(0.850)    // Direct density specification in g/cm³
    .withAverageBoilingPointCelsius(250.0);

System.out.println("API Gravity Conversion Demonstration:");
System.out.println("Cut with API gravity 35.0°:");
System.out.println("  Resolved density: " + String.format("%.3f", apiGravityCut.resolveDensity()) + " g/cm³");

System.out.println("\nCut with direct density 0.850 g/cm³:");
System.out.println("  Density: " + String.format("%.3f", densityCut.resolveDensity()) + " g/cm³");

// Show the relationship between API gravity and density
System.out.println("\nAPI Gravity Reference:");
System.out.println("  > 31.1° API = Light crude oil");
System.out.println("  22.3° - 31.1° API = Medium crude oil");
System.out.println("  10° - 22.3° API = Heavy crude oil");
System.out.println("  < 10° API = Extra heavy crude oil");

// Calculate API gravity from density (reverse calculation)
double density = 0.850; // g/cm³
double specificGravity = density / 0.999016; // Relative to water at 60°F
double calculatedAPI = (141.5 / specificGravity) - 131.5;
System.out.println("\nReverse calculation:");
System.out.println("Density " + density + " g/cm³ = " + String.format("%.1f", calculatedAPI) + "° API");

## Setting Boiling Points in Different Units

The class provides convenient methods for setting boiling points in Kelvin, Celsius, and Fahrenheit. This flexibility accommodates different data sources and regional preferences.

In [ ]:
// Demonstrate different temperature units for boiling points
AssayCut kelvinCut = new AssayCut("Kelvin Temperature")
    .withMassFraction(0.25)
    .withApiGravity(40.0)
    .withAverageBoilingPointKelvin(473.15);  // 200°C

AssayCut celsiusCut = new AssayCut("Celsius Temperature")
    .withMassFraction(0.25)
    .withApiGravity(40.0)
    .withAverageBoilingPointCelsius(200.0);  // Same as above

AssayCut fahrenheitCut = new AssayCut("Fahrenheit Temperature")
    .withMassFraction(0.25)
    .withApiGravity(40.0)
    .withAverageBoilingPointFahrenheit(392.0);  // Same as above (200°C = 392°F)

AssayCut mixedUnitsCut = new AssayCut("Mixed Units Example")
    .withWeightPercent(25.0)      // Percentage form
    .withApiGravity(40.0)         // API gravity
    .withAverageBoilingPointFahrenheit(752.0);  // 400°C in Fahrenheit

System.out.println("Temperature Unit Conversion Demonstration:");
System.out.println("All cuts should have the same resolved boiling point:");
System.out.println("Kelvin cut: " + String.format("%.2f", kelvinCut.resolveAverageBoilingPoint()) + " K");
System.out.println("Celsius cut: " + String.format("%.2f", celsiusCut.resolveAverageBoilingPoint()) + " K");
System.out.println("Fahrenheit cut: " + String.format("%.2f", fahrenheitCut.resolveAverageBoilingPoint()) + " K");
System.out.println("Mixed units cut: " + String.format("%.2f", mixedUnitsCut.resolveAverageBoilingPoint()) + " K");

// Convert back to Celsius for verification
double tempCelsius = kelvinCut.resolveAverageBoilingPoint() - 273.15;
System.out.println("\nVerification: " + String.format("%.1f", tempCelsius) + "°C");

## Applying Assay Characterization to the System

Now let's create an `OilAssayCharacterisation` object and apply our petroleum fractions to the thermodynamic system. This step converts the assay cuts into pseudo-components that can be used for thermodynamic calculations.

In [ ]:
// Create a new thermodynamic system for this example
SystemInterface assaySystem = new SystemSrkEos(288.15, 1.01325);
assaySystem.setMixingRule("classic");
assaySystem.createDatabase(true);

// Create the oil assay characterization object
OilAssayCharacterisation oilAssay = new OilAssayCharacterisation(assaySystem);

// Set total assay mass (1 kg basis for calculations)
oilAssay.setTotalAssayMass(1.0);

// Add the cuts we created earlier (using the mass fraction cuts)
oilAssay.addCut(lightFraction);
oilAssay.addCut(mediumFraction);
oilAssay.addCut(heavyFraction);
oilAssay.addCut(residueFraction);

System.out.println("Oil assay characterization setup:");
System.out.println("Number of cuts: " + oilAssay.getCuts().size());
System.out.println("Total assay mass: " + oilAssay.getTotalAssayMass() + " kg");

// Apply the characterization to the system
System.out.println("\nApplying assay characterization...");
oilAssay.apply();

System.out.println("System after characterization:");
System.out.println("Number of components: " + assaySystem.getNumberOfComponents());
for (int i = 0; i < assaySystem.getNumberOfComponents(); i++) {
    String componentName = assaySystem.getComponent(i).getComponentName();
    double moles = assaySystem.getComponent(i).getNumberOfMolesInPhase();
    double molarMass = assaySystem.getComponent(i).getMolarMass() * 1000; // Convert to g/mol
    System.out.println("  " + componentName + ": " + String.format("%.4f", moles) + " mol, MW = " + String.format("%.1f", molarMass) + " g/mol");
}

## Advanced Example: Complex Multi-Cut Assay

Let's create a more realistic petroleum assay with multiple cuts representing a typical North Sea crude oil similar to those characterized in Equinor's assay reports.

In [ ]:
// Create a realistic North Sea crude oil assay
SystemInterface northSeaSystem = new SystemSrkEos(288.15, 1.01325);
northSeaSystem.setMixingRule("classic");
northSeaSystem.createDatabase(true);

OilAssayCharacterisation northSeaAssay = new OilAssayCharacterisation(northSeaSystem);

// Create detailed cuts based on typical North Sea crude characteristics
List<AssayCut> northSeaCuts = new ArrayList<>();

// Light ends (C5-C7)
northSeaCuts.add(new AssayCut("Light Ends")
    .withWeightPercent(3.5)
    .withApiGravity(85.0)
    .withAverageBoilingPointCelsius(50.0));

// Light Naphtha (C7-C9)
northSeaCuts.add(new AssayCut("Light Naphtha")
    .withWeightPercent(8.2)
    .withApiGravity(72.0)
    .withAverageBoilingPointCelsius(95.0));

// Heavy Naphtha (C9-C12)
northSeaCuts.add(new AssayCut("Heavy Naphtha")
    .withWeightPercent(12.8)
    .withApiGravity(58.0)
    .withAverageBoilingPointCelsius(140.0));

// Kerosene (C12-C16)
northSeaCuts.add(new AssayCut("Kerosene")
    .withWeightPercent(15.5)
    .withApiGravity(48.0)
    .withAverageBoilingPointCelsius(210.0));

// Light Gas Oil (C16-C20)
northSeaCuts.add(new AssayCut("Light Gas Oil")
    .withWeightPercent(18.0)
    .withApiGravity(38.0)
    .withAverageBoilingPointCelsius(290.0));

// Heavy Gas Oil (C20-C30)
northSeaCuts.add(new AssayCut("Heavy Gas Oil")
    .withWeightPercent(22.5)
    .withApiGravity(28.0)
    .withAverageBoilingPointCelsius(380.0));

// Vacuum Gas Oil (C30-C50)
northSeaCuts.add(new AssayCut("Vacuum Gas Oil")
    .withWeightPercent(12.8)
    .withApiGravity(20.0)
    .withAverageBoilingPointCelsius(480.0));

// Residue (C50+)
northSeaCuts.add(new AssayCut("Residue")
    .withWeightPercent(6.7)
    .withApiGravity(12.0)
    .withAverageBoilingPointCelsius(650.0));

// Add all cuts to the assay
northSeaAssay.addCuts(northSeaCuts);

System.out.println("North Sea Crude Oil Assay:");
System.out.println("=============================");
double totalWeight = 0.0;
for (AssayCut cut : northSeaAssay.getCuts()) {
    double weight = cut.getMassFraction() * 100;
    totalWeight += weight;
    System.out.println(String.format("%-18s: %5.1f%% wt, %5.1f° API, %6.1f°C",
        cut.getName(), weight,
        // Calculate API from density for display
        (141.5 / (cut.resolveDensity() / 0.999016)) - 131.5,
        cut.resolveAverageBoilingPoint() - 273.15));
}
System.out.println("                   ------");
System.out.println(String.format("%-18s: %5.1f%% wt", "Total", totalWeight));

// Apply the characterization
northSeaAssay.apply();
System.out.println("\nCharacterization complete. System now contains " +
                   northSeaSystem.getNumberOfComponents() + " pseudo-components.");

## Handling Mixed Mass and Volume Fraction Data

In practice, assay data often comes from different sources with mixed units. The class can handle scenarios where some cuts have mass fractions and others have volume fractions.

In [ ]:
// Demonstrate mixed mass and volume fraction handling
SystemInterface mixedSystem = new SystemSrkEos(288.15, 1.01325);
mixedSystem.setMixingRule("classic");
mixedSystem.createDatabase(true);

OilAssayCharacterisation mixedAssay = new OilAssayCharacterisation(mixedSystem);

// Some cuts defined by mass fraction (from laboratory analysis)
AssayCut massCut1 = new AssayCut("Known Mass Cut 1")
    .withMassFraction(0.15)  // 15% by mass - precisely known
    .withApiGravity(60.0)
    .withAverageBoilingPointCelsius(80.0);

AssayCut massCut2 = new AssayCut("Known Mass Cut 2")
    .withMassFraction(0.25)  // 25% by mass - precisely known
    .withApiGravity(45.0)
    .withAverageBoilingPointCelsius(180.0);

// Other cuts defined by volume fraction (from distillation data)
AssayCut volumeCut1 = new AssayCut("Distillation Cut 1")
    .withVolumePercent(35.0)  // 35% by volume from distillation
    .withDensity(0.85)
    .withAverageBoilingPointCelsius(320.0);

AssayCut volumeCut2 = new AssayCut("Distillation Cut 2")
    .withVolumePercent(25.0)  // 25% by volume from distillation
    .withDensity(0.95)
    .withAverageBoilingPointCelsius(450.0);

// Add all cuts
mixedAssay.addCut(massCut1);
mixedAssay.addCut(massCut2);
mixedAssay.addCut(volumeCut1);
mixedAssay.addCut(volumeCut2);

System.out.println("Mixed Fraction Data Example:");
System.out.println("=============================");
System.out.println("Mass-based cuts (known precisely):");
System.out.println("  " + massCut1.getName() + ": " + (massCut1.getMassFraction() * 100) + "% mass");
System.out.println("  " + massCut2.getName() + ": " + (massCut2.getMassFraction() * 100) + "% mass");

System.out.println("\nVolume-based cuts (from distillation):");
System.out.println("  " + volumeCut1.getName() + ": " + (volumeCut1.getVolumeFraction() * 100) + "% volume");
System.out.println("  " + volumeCut2.getName() + ": " + (volumeCut2.getVolumeFraction() * 100) + "% volume");

// Apply characterization - the class will automatically convert volume fractions to mass fractions
mixedAssay.apply();

System.out.println("\nCharacterization applied successfully!");
System.out.println("The class automatically:")
System.out.println("1. Used the specified mass fractions (40% total)");
System.out.println("2. Converted volume fractions to mass fractions for remaining 60%");
System.out.println("3. Normalized to ensure total mass fraction = 1.0");
System.out.println("\nResulting system has " + mixedSystem.getNumberOfComponents() + " components.");

## Error Handling and Validation

The `OilAssayCharacterisation` class includes comprehensive validation to catch common errors in assay data. Let's demonstrate some of these validation features.

In [ ]:
// Demonstrate error handling and validation
System.out.println("Error Handling and Validation Examples:");
System.out.println("=======================================");

// Example 1: Negative fraction values
try {
    AssayCut invalidCut = new AssayCut("Invalid Cut")
        .withMassFraction(-0.1);  // Negative fraction
} catch (IllegalArgumentException e) {
    System.out.println("✓ Caught negative fraction error: " + e.getMessage());
}

// Example 2: Missing density information
try {
    AssayCut noDensityCut = new AssayCut("No Density Cut")
        .withMassFraction(0.2)
        .withAverageBoilingPointCelsius(200.0);
    double density = noDensityCut.resolveDensity();  // This will fail
} catch (IllegalStateException e) {
    System.out.println("✓ Caught missing density error: " + e.getMessage());
}

// Example 3: Missing boiling point
try {
    AssayCut noBoilingPointCut = new AssayCut("No BP Cut")
        .withMassFraction(0.2)
        .withApiGravity(35.0);
    double bp = noBoilingPointCut.resolveAverageBoilingPoint();  // This will fail
} catch (IllegalStateException e) {
    System.out.println("✓ Caught missing boiling point error: " + e.getMessage());
}

// Example 4: Mass fractions exceeding unity
SystemInterface validationSystem = new SystemSrkEos(288.15, 1.01325);
validationSystem.setMixingRule("classic");
validationSystem.createDatabase(true);

OilAssayCharacterisation invalidAssay = new OilAssayCharacterisation(validationSystem);

try {
    invalidAssay.addCut(new AssayCut("Cut 1")
        .withMassFraction(0.6)
        .withApiGravity(35.0)
        .withAverageBoilingPointCelsius(200.0));

    invalidAssay.addCut(new AssayCut("Cut 2")
        .withMassFraction(0.6)  // Total = 1.2 > 1.0
        .withApiGravity(30.0)
        .withAverageBoilingPointCelsius(300.0));

    invalidAssay.apply();  // This will fail
} catch (IllegalStateException e) {
    System.out.println("✓ Caught excess mass fraction error: " + e.getMessage());
}

// Example 5: Valid percentage input handling
try {
    AssayCut percentCut = new AssayCut("Percent Cut")
        .withWeightPercent(120.0);  // > 100% - should be treated as percentage
    System.out.println("✓ Automatic percentage conversion: 120% → " +
                       String.format("%.1f", percentCut.getMassFraction() * 100) + "%");
} catch (Exception e) {
    System.out.println("Percentage handling: " + e.getMessage());
}

System.out.println("\nValidation Summary:");
System.out.println("- Negative fractions are rejected");
System.out.println("- Missing required properties are detected");
System.out.println("- Mass balance is enforced");
System.out.println("- Percentage values are auto-converted when sensible");

## Cloning and Modifying Assay Objects

The class supports cloning, which is useful for creating variations of assay characterizations or for what-if analysis scenarios.

In [ ]:
// Demonstrate cloning functionality for what-if scenarios
SystemInterface baseSystem = new SystemSrkEos(288.15, 1.01325);
baseSystem.setMixingRule("classic");
baseSystem.createDatabase(true);

// Create base assay
OilAssayCharacterisation baseAssay = new OilAssayCharacterisation(baseSystem);
baseAssay.addCut(new AssayCut("Light Fraction")
    .withMassFraction(0.3)
    .withApiGravity(55.0)
    .withAverageBoilingPointCelsius(120.0));

baseAssay.addCut(new AssayCut("Medium Fraction")
    .withMassFraction(0.4)
    .withApiGravity(35.0)
    .withAverageBoilingPointCelsius(250.0));

baseAssay.addCut(new AssayCut("Heavy Fraction")
    .withMassFraction(0.3)
    .withApiGravity(20.0)
    .withAverageBoilingPointCelsius(400.0));

System.out.println("Cloning and Modification Example:");
System.out.println("=================================");
System.out.println("Base assay has " + baseAssay.getCuts().size() + " cuts");

// Clone the assay for modification
OilAssayCharacterisation modifiedAssay = baseAssay.clone();

// Create a new system for the cloned assay
SystemInterface modifiedSystem = new SystemSrkEos(288.15, 1.01325);
modifiedSystem.setMixingRule("classic");
modifiedSystem.createDatabase(true);
modifiedAssay.setThermoSystem(modifiedSystem);

System.out.println("Cloned assay has " + modifiedAssay.getCuts().size() + " cuts");

// Clear and modify the cloned assay
modifiedAssay.clearCuts();
modifiedAssay.addCut(new AssayCut("Modified Light")
    .withMassFraction(0.35)  // Increased light fraction
    .withApiGravity(60.0)    // Higher API gravity
    .withAverageBoilingPointCelsius(110.0));

modifiedAssay.addCut(new AssayCut("Modified Medium")
    .withMassFraction(0.35)  // Adjusted
    .withApiGravity(35.0)
    .withAverageBoilingPointCelsius(250.0));

modifiedAssay.addCut(new AssayCut("Modified Heavy")
    .withMassFraction(0.30)  // Reduced heavy fraction
    .withApiGravity(25.0)    // Slightly lighter
    .withAverageBoilingPointCelsius(380.0));

// Apply both characterizations
baseAssay.apply();
modifiedAssay.apply();

System.out.println("\nComparison of base vs modified assay:");
System.out.println("Base assay components: " + baseSystem.getNumberOfComponents());
System.out.println("Modified assay components: " + modifiedSystem.getNumberOfComponents());

// Demonstrate that original assay is unchanged
System.out.println("\nOriginal assay cuts (unchanged):");
for (AssayCut cut : baseAssay.getCuts()) {
    System.out.println("  " + cut.getName() + ": " +
                       String.format("%.1f", cut.getMassFraction() * 100) + "% mass");
}

System.out.println("\nModified assay cuts:");
for (AssayCut cut : modifiedAssay.getCuts()) {
    System.out.println("  " + cut.getName() + ": " +
                       String.format("%.1f", cut.getMassFraction() * 100) + "% mass");
}

System.out.println("\n✓ Cloning allows independent modification of assay objects");

## Summary and Best Practices

The `OilAssayCharacterisation` class provides a robust framework for converting petroleum assay data into thermodynamic models suitable for process simulation. Here are the key takeaways:

### Key Features
- **Flexible Input**: Accepts mass fractions, volume fractions, weight percentages, or volume percentages
- **Multiple Density Formats**: Direct density or API gravity specification
- **Temperature Units**: Kelvin, Celsius, or Fahrenheit for boiling points  
- **Automatic Conversions**: Handles unit conversions and mass balance normalization
- **Robust Validation**: Comprehensive error checking for data consistency
- **Cloning Support**: Enables what-if analysis and scenario comparison

### Best Practices
1. **Always specify either density or API gravity** for each cut
2. **Provide boiling point data** unless you have explicit molar mass values
3. **Ensure mass/volume fractions sum to ≤ 1.0** to avoid validation errors
4. **Use consistent temperature units** within your workflow
5. **Apply characterization only after all cuts are defined**
6. **Clone assay objects** for sensitivity analysis or alternative scenarios

### Integration with NeqSim
- The characterized system can be used directly in NeqSim process equipment
- Pseudo-components are automatically added as TBP (True Boiling Point) fractions
- The system is ready for thermodynamic calculations (flash, distillation, etc.)

### References
- [Equinor Crude Oil Assay Update (October 2025)](https://www.equinor.com/news/crude-oil-assay-update-10-10-2025)
- NeqSim documentation: [docs/wiki](../index.md)
- API gravity standard: API 2540 - Manual of Petroleum Measurement Standards

This concludes the demonstration of the `OilAssayCharacterisation` class. The class provides a systematic and robust approach to petroleum fluid characterization that integrates seamlessly with NeqSim's thermodynamic modeling capabilities.

## Practical Example: Equinor Johan Sverdrup Crude Oil Phase Envelope

Let's demonstrate a practical application using actual assay data from Equinor's Johan Sverdrup crude oil (based on publicly available assay information) to create a NeqSim fluid and calculate its phase envelope. This example shows how the `OilAssayCharacterisation` class integrates with NeqSim's thermodynamic capabilities.

In [ ]:
// Import additional classes for phase envelope calculations
import neqsim.thermodynamicoperations.ThermodynamicOperations;
import neqsim.thermo.system.SystemPrEos;

// Create a system for Johan Sverdrup crude oil characterization
// Using PR-EOS which is often preferred for phase envelope calculations
SystemInterface johanSverdrupSystem = new SystemPrEos(288.15, 1.01325);
johanSverdrupSystem.setMixingRule("classic");
johanSverdrupSystem.createDatabase(true);

// Add light components typically found in Johan Sverdrup crude
// Based on typical North Sea crude gas content
johanSverdrupSystem.addComponent("methane", 0.005);      // 0.5 mol% - dissolved gas
johanSverdrupSystem.addComponent("ethane", 0.003);       // 0.3 mol%
johanSverdrupSystem.addComponent("propane", 0.004);      // 0.4 mol%
johanSverdrupSystem.addComponent("i-butane", 0.002);     // 0.2 mol%
johanSverdrupSystem.addComponent("n-butane", 0.003);     // 0.3 mol%
johanSverdrupSystem.addComponent("i-pentane", 0.002);    // 0.2 mol%
johanSverdrupSystem.addComponent("n-pentane", 0.002);    // 0.2 mol%

// Create oil assay characterization for Johan Sverdrup crude
OilAssayCharacterisation johanSverdrupAssay = new OilAssayCharacterisation(johanSverdrupSystem);

// Johan Sverdrup crude oil assay data (typical values based on public information)
// This is a high-quality, light crude oil from the North Sea
List<AssayCut> johanSverdrupCuts = new ArrayList<>();

// Light petroleum fractions
johanSverdrupCuts.add(new AssayCut("C6-C7")
    .withWeightPercent(4.2)
    .withApiGravity(78.0)
    .withAverageBoilingPointCelsius(75.0));

johanSverdrupCuts.add(new AssayCut("Light Naphtha C7-C9")
    .withWeightPercent(9.8)
    .withApiGravity(65.0)
    .withAverageBoilingPointCelsius(105.0));

johanSverdrupCuts.add(new AssayCut("Heavy Naphtha C9-C12")
    .withWeightPercent(14.5)
    .withApiGravity(55.0)
    .withAverageBoilingPointCelsius(145.0));

johanSverdrupCuts.add(new AssayCut("Kerosene C12-C16")
    .withWeightPercent(16.8)
    .withApiGravity(46.0)
    .withAverageBoilingPointCelsius(215.0));

johanSverdrupCuts.add(new AssayCut("Light Gas Oil C16-C20")
    .withWeightPercent(19.2)
    .withApiGravity(37.0)
    .withAverageBoilingPointCelsius(295.0));

johanSverdrupCuts.add(new AssayCut("Heavy Gas Oil C20-C30")
    .withWeightPercent(20.5)
    .withApiGravity(29.0)
    .withAverageBoilingPointCelsius(385.0));

johanSverdrupCuts.add(new AssayCut("Vacuum Gas Oil C30-C50")
    .withWeightPercent(11.8)
    .withApiGravity(22.0)
    .withAverageBoilingPointCelsius(485.0));

johanSverdrupCuts.add(new AssayCut("Residue C50+")
    .withWeightPercent(3.2)
    .withApiGravity(15.0)
    .withAverageBoilingPointCelsius(650.0));

// Add all cuts to the assay
johanSverdrupAssay.addCuts(johanSverdrupCuts);

System.out.println("Johan Sverdrup Crude Oil Assay (Representative Data):");
System.out.println("=====================================================");
System.out.println("Overall API Gravity: ~41° API (light crude oil)");
System.out.println("Sulfur Content: <0.1% (sweet crude)");
System.out.println();

double totalWeight = 0.0;
for (AssayCut cut : johanSverdrupAssay.getCuts()) {
    double weight = cut.getMassFraction() * 100;
    totalWeight += weight;
    double apiGravity = (141.5 / (cut.resolveDensity() / 0.999016)) - 131.5;
    System.out.println(String.format("%-22s: %5.1f%% wt, %5.1f° API, %6.1f°C",
        cut.getName(), weight, apiGravity,
        cut.resolveAverageBoilingPoint() - 273.15));
}
System.out.println("                        ------");
System.out.println(String.format("%-22s: %5.1f%% wt", "Petroleum Fractions", totalWeight));

// Apply the characterization
johanSverdrupAssay.apply();

System.out.println("\nSystem composition after characterization:");
System.out.println("Light components: " + 7 + " components");
System.out.println("Petroleum fractions: " + johanSverdrupCuts.size() + " pseudo-components");
System.out.println("Total components: " + johanSverdrupSystem.getNumberOfComponents());

### Calculate Phase Envelope

Now let's calculate the phase envelope for this Johan Sverdrup crude oil system. The phase envelope shows the pressure-temperature conditions where vapor and liquid phases coexist, which is crucial for reservoir engineering, production optimization, and process design.

In [ ]:
// Calculate phase envelope for Johan Sverdrup crude oil
System.out.println("Calculating Phase Envelope for Johan Sverdrup Crude Oil:");
System.out.println("========================================================");

// Create thermodynamic operations object
ThermodynamicOperations thermoOps = new ThermodynamicOperations(johanSverdrupSystem);

// Set up for phase envelope calculation
johanSverdrupSystem.setTemperature(273.15 + 20.0);  // Start at 20°C
johanSverdrupSystem.setPressure(1.01325);           // Start at 1 bar

try {
    // Calculate phase envelope
    System.out.println("Starting phase envelope calculation...");
    thermoOps.calcPTphaseEnvelope();

    System.out.println("✓ Phase envelope calculation completed successfully!");

    // Get phase envelope data
    double[][] phaseEnvelopeData = thermoOps.get("phaseEnvelope");

    if (phaseEnvelopeData != null && phaseEnvelopeData.length > 0) {
        System.out.println("\nPhase Envelope Results:");
        System.out.println("Number of calculated points: " + phaseEnvelopeData.length);

        // Find critical point (highest temperature on bubble point curve)
        double maxTemp = 0.0;
        double criticalPressure = 0.0;
        double minPressure = Double.MAX_VALUE;
        double maxPressure = 0.0;

        System.out.println("\nSample Phase Envelope Points (T[°C], P[bar]):");
        System.out.println("Temperature[°C]  Pressure[bar]  Phase");
        System.out.println("----------------------------------------");

        for (int i = 0; i < Math.min(10, phaseEnvelopeData.length); i++) {
            double temp = phaseEnvelopeData[i][0] - 273.15;  // Convert K to °C
            double pressure = phaseEnvelopeData[i][1];        // Already in bar

            if (phaseEnvelopeData[i][0] > maxTemp) {
                maxTemp = phaseEnvelopeData[i][0];
                criticalPressure = pressure;
            }

            minPressure = Math.min(minPressure, pressure);
            maxPressure = Math.max(maxPressure, pressure);

            String phase = (i < phaseEnvelopeData.length / 2) ? "Bubble Point" : "Dew Point";
            System.out.println(String.format("%12.1f    %11.2f  %s", temp, pressure, phase));
        }

        System.out.println("\nPhase Envelope Summary:");
        System.out.println("Critical Temperature: " + String.format("%.1f", maxTemp - 273.15) + "°C");
        System.out.println("Critical Pressure: " + String.format("%.1f", criticalPressure) + " bar");
        System.out.println("Pressure Range: " + String.format("%.2f", minPressure) + " - " + String.format("%.1f", maxPressure) + " bar");

        // Practical implications
        System.out.println("\nPractical Implications for Johan Sverdrup:");
        System.out.println("- Reservoir conditions: ~150°C, ~250 bar (single phase oil)");
        System.out.println("- Surface separation: ~60°C, ~20 bar (optimal for gas-oil separation)");
        System.out.println("- Export conditions: ~15°C, ~1 bar (stable liquid phase)");

    } else {
        System.out.println("Phase envelope data not available in expected format.");

        // Alternative: Calculate bubble point at standard conditions
        johanSverdrupSystem.setTemperature(273.15 + 60.0);  // 60°C
        thermoOps.bubblePointPressureFlash(false);

        System.out.println("\nBubble Point Calculation at 60°C:");
        System.out.println("Bubble point pressure: " + String.format("%.2f", johanSverdrupSystem.getPressure()) + " bar");

        // Calculate dew point
        johanSverdrupSystem.setTemperature(273.15 + 60.0);
        thermoOps.dewPointPressureFlash();

        System.out.println("Dew point pressure: " + String.format("%.2f", johanSverdrupSystem.getPressure()) + " bar");
    }

} catch (Exception e) {
    System.out.println("Error during phase envelope calculation: " + e.getMessage());
    System.out.println("This might be due to system complexity or numerical issues.");

    // Fallback: Calculate some basic phase behavior properties
    System.out.println("\nFallback: Basic Phase Behavior Analysis");

    try {
        // Set typical separator conditions
        johanSverdrupSystem.setTemperature(273.15 + 60.0);  // 60°C
        johanSverdrupSystem.setPressure(20.0);              // 20 bar

        thermoOps.TPflash();
        johanSverdrupSystem.display();

        System.out.println("\nAt separator conditions (60°C, 20 bar):");
        System.out.println("Number of phases: " + johanSverdrupSystem.getNumberOfPhases());
        if (johanSverdrupSystem.getNumberOfPhases() > 1) {
            System.out.println("Gas phase fraction: " + String.format("%.3f", johanSverdrupSystem.getPhase(0).getBeta()));
            System.out.println("Oil phase fraction: " + String.format("%.3f", johanSverdrupSystem.getPhase(1).getBeta()));
        }

    } catch (Exception e2) {
        System.out.println("Could not complete basic flash calculation: " + e2.getMessage());
    }
}

### Analysis and Engineering Applications

The phase envelope calculation provides critical information for petroleum engineering applications:

In [ ]:
// Demonstrate practical applications of the characterized fluid
System.out.println("Engineering Applications of Johan Sverdrup Fluid Model:");
System.out.println("======================================================");

// 1. Reservoir Conditions Analysis
System.out.println("\n1. RESERVOIR CONDITIONS ANALYSIS");
System.out.println("--------------------------------");
johanSverdrupSystem.setTemperature(273.15 + 150.0);  // Typical reservoir temp: 150°C
johanSverdrupSystem.setPressure(250.0);              // Typical reservoir pressure: 250 bar

try {
    thermoOps.TPflash();

    System.out.println("Reservoir conditions (150°C, 250 bar):");
    System.out.println("Number of phases: " + johanSverdrupSystem.getNumberOfPhases());
    System.out.println("Fluid state: " + (johanSverdrupSystem.getNumberOfPhases() == 1 ? "Single phase oil" : "Two phase"));

    if (johanSverdrupSystem.getNumberOfPhases() == 1) {
        System.out.println("Density: " + String.format("%.1f", johanSverdrupSystem.getDensity()) + " kg/m³");
        System.out.println("Viscosity: " + String.format("%.2e", johanSverdrupSystem.getViscosity()) + " Pa·s");
    }
} catch (Exception e) {
    System.out.println("Could not flash at reservoir conditions: " + e.getMessage());
}

// 2. Production Separator Analysis
System.out.println("\n2. PRODUCTION SEPARATOR ANALYSIS");
System.out.println("---------------------------------");
johanSverdrupSystem.setTemperature(273.15 + 60.0);   // Separator temp: 60°C
johanSverdrupSystem.setPressure(20.0);               // Separator pressure: 20 bar

try {
    thermoOps.TPflash();

    System.out.println("Separator conditions (60°C, 20 bar):");
    System.out.println("Number of phases: " + johanSverdrupSystem.getNumberOfPhases());

    if (johanSverdrupSystem.getNumberOfPhases() > 1) {
        double gasPhase = johanSverdrupSystem.getPhase(0).getBeta();
        double oilPhase = johanSverdrupSystem.getPhase(1).getBeta();

        System.out.println("Gas phase fraction: " + String.format("%.1f", gasPhase * 100) + "%");
        System.out.println("Oil phase fraction: " + String.format("%.1f", oilPhase * 100) + "%");

        System.out.println("Gas density: " + String.format("%.1f", johanSverdrupSystem.getPhase(0).getDensity()) + " kg/m³");
        System.out.println("Oil density: " + String.format("%.1f", johanSverdrupSystem.getPhase(1).getDensity()) + " kg/m³");

        // Calculate GOR (Gas-Oil Ratio)
        double gasVolume = johanSverdrupSystem.getPhase(0).getCorrectedVolume();
        double oilVolume = johanSverdrupSystem.getPhase(1).getCorrectedVolume();
        double GOR = gasVolume / oilVolume;  // m³/m³

        System.out.println("Gas-Oil Ratio (GOR): " + String.format("%.0f", GOR) + " m³/m³");
    }
} catch (Exception e) {
    System.out.println("Could not flash at separator conditions: " + e.getMessage());
}

// 3. Export/Storage Conditions
System.out.println("\n3. EXPORT/STORAGE CONDITIONS ANALYSIS");
System.out.println("--------------------------------------");
johanSverdrupSystem.setTemperature(273.15 + 15.0);   // Storage temp: 15°C
johanSverdrupSystem.setPressure(1.01325);            // Atmospheric pressure

try {
    thermoOps.TPflash();

    System.out.println("Storage conditions (15°C, 1 bar):");
    System.out.println("Number of phases: " + johanSverdrupSystem.getNumberOfPhases());

    if (johanSverdrupSystem.getNumberOfPhases() == 1) {
        System.out.println("Stable single phase liquid");
        System.out.println("Oil density: " + String.format("%.1f", johanSverdrupSystem.getDensity()) + " kg/m³");

        // Calculate API gravity at storage conditions
        double specificGravity = johanSverdrupSystem.getDensity() / 999.016;  // Relative to water at 60°F
        double apiGravity = (141.5 / specificGravity) - 131.5;
        System.out.println("API gravity: " + String.format("%.1f", apiGravity) + "°");

        System.out.println("Viscosity: " + String.format("%.2e", johanSverdrupSystem.getViscosity()) + " Pa·s");
    } else {
        System.out.println("Warning: Multiple phases at storage conditions");
    }
} catch (Exception e) {
    System.out.println("Could not flash at storage conditions: " + e.getMessage());
}

// 4. Process Design Insights
System.out.println("\n4. PROCESS DESIGN INSIGHTS");
System.out.println("---------------------------");
System.out.println("✓ The characterized Johan Sverdrup crude oil model can be used for:");
System.out.println("  - Distillation column design and optimization");
System.out.println("  - Heat exchanger sizing and thermal analysis");
System.out.println("  - Separator vessel design and operation");
System.out.println("  - Pipeline flow assurance studies");
System.out.println("  - Storage tank design and vapor loss calculations");
System.out.println("  - Refinery planning and product yield estimation");

System.out.println("\n✓ Key advantages of assay-based characterization:");
System.out.println("  - Realistic representation of petroleum fractions");
System.out.println("  - Accurate boiling point distribution");
System.out.println("  - Proper density and molecular weight correlations");
System.out.println("  - Integration with standard petroleum engineering workflows");